In [ ]:
from google.cloud import language_v1
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

def sample_analyze_entities(gcs_content_uri):
    client = language_v1.LanguageServiceClient()
    type_ = language_v1.Document.Type.PLAIN_TEXT

    language = "en"
    document = {"gcs_content_uri": gcs_content_uri, "type_": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_entities(request = {'document': document, 'encoding_type': encoding_type})
    # Loop through entitites returned from the API
    for entity in response.entities:
        if language_v1.Entity.Type(entity.type_).name == 'OTHER': 
            print(u"Entity Name : {}".format(entity.name))

gcs_content_uri = 'gs://lca_storage/seed_data4.txt'
sample_analyze_entities(gcs_content_uri)

In [ ]:
import pandas as pd
import nltk
import spacy
from tqdm import tqdm

seed_file ='./cuad-data/seed_data copy.csv'
nlp = spacy.load("en_core_web_sm")

def create_tokens(dataframe):
    tokens = []
    for doc in tqdm(nlp.pipe(dataframe)):        
        if doc.is_parsed:
            tokens.append([n.lemma_.lower() for n in doc if (not n.is_punct and not n.is_space and not n.is_stop)])
        else:
            tokens.append("")
    return tokens

raw = pd.read_csv(seed_file)
question_tokens = create_tokens(raw['Statements'])

''' print (raw['Statements'])
for rwo in raw['Statements']: 
    question_tokens = create_tokens(raw['Statements'])
    '''

In [ ]:
import textacy
import textacy

text = " ".join(raw.tolist())
nlp = spacy.load('en_core_web_sm')
nlp.max_length = len(text)
keywords = []

for tokenlist in tqdm(question_tokens):
    doc = nlp(" ".join(tokenlist))
    extract = textacy.ke.sgrank(doc, ngrams=(1), window_size=2, normalize=None, topn = 2, include_pos=['NOUN', 'PROPN'])
    for a, b in extract:
        keywords.append(a)

res = sorted(set(keywords), key = lambda x: keywords.count(x), reverse=True)
